## Tooling - GSW standalone board

In [1]:
import Switch as sw

module ser02.py: list_ports(), open_port("COMx"), r(addr), w(addr, value), close_ports()
device: /dev/ttyS0
name: ttyS0
description: n/a
device_path: /sys/devices/platform/soc/fe215040.serial
hwid: n/a
subsystem: platform

device: /dev/ttyUSB0
name: ttyUSB0
description: FT232R USB UART
device_path: /sys/devices/platform/scb/fd500000.pcie/pci0000:00/0000:00:00.0/0000:01:00.0/usb1/1-1/1-1.3/1-1.3:1.0/ttyUSB0
hwid: USB VID:PID=0403:6001 SER=A503TT9Q LOCATION=1-1.3
subsystem: usb-serial

device: /dev/ttyAMA1
name: ttyAMA1
description: ttyAMA1
device_path: /sys/devices/platform/soc/fe201600.serial
hwid: fe201600.serial
subsystem: amba

device: /dev/ttyAMA0
name: ttyAMA0
description: ttyAMA0
device_path: /sys/devices/platform/soc/fe201000.serial
hwid: fe201000.serial
subsystem: amba

/dev/ttyS0 - n/a
/dev/ttyUSB0 - FT232R USB UART
/dev/ttyAMA1 - ttyAMA1
/dev/ttyAMA0 - ttyAMA0
Please input the port name (e.g. COMx): /dev/ttyUSB0
Open serial port at: /dev/ttyUSB0
r(0xfa10) = 0x1713
r(0xfa11) =

In [3]:
import sys
print (224*0x1000000)
print ("%#x"%(224*0x1000000))
print ("%#x"%sys.maxsize)

3758096384
0xe0000000
0x7fffffff


#### Check if microcode enabled
After microcode download, reg0x456 = 0x0008 (**the parser microcode is valid**), reg0x457 = 0x0201 (**bit1=0 PCE is enabled**).

Without microcode downloaded, reg0x456 = 0x0000, reg0x457 = 0x0001

0x456 bit 3
<img src="GSW_reg0x456_bit3.PNG">

In [3]:
print (hex(sw.r(0xf383)))
print (hex(sw.r(0xf384)))
print (hex(sw.r(0x456)))
print (hex(sw.r(0x457)))

0x3c3
0x3f
0x0
0x1


#### Download microcode
It takes 25sec.

In [3]:
sw.pyGSW_HW_Init()
print ("finished.")

finished.


#### List of capabilities

In [3]:
ret = {"nCapType": "0"}
a = 0
while ret["nCapType"]!="13":
    ret = sw.pyGSW_CapGet(a, pr=0)
    #print ( "nCapType", ret["nCapType"])
    desc = ret["cDesc[128]"][2:-1].split()
    cap = ""
    for s in desc:
        if (s == "0x00"):
            break
        else:
            cap = cap + (bytes.fromhex(s[2:]).decode('utf-8'))
    print (cap, ret["nCap"])
    a = a+1

Number of Ethernet ports 7
Number of virtual ports 6
Pcket buffer size[in Bytes]: 131072
Buffer Segment size: 256
Number of queues: 32
Number of traffic meters: 16
Number of rate shapers: 32
Number of VLAN groups: 64
Number of FIDs: 64
Number of MAC entries: 2048
Number of multicast entries 64
Number of PPPoE sessions: 16
Number of STAG VLAN groups: 64
 0


In [9]:
ret = sw.pyGSW_MAC_TableEntryRead(1, pr = 1)
print ("ret = ", sw.nRet[ret["ret"]])
for a in range(100):
    print ("nMAC[6] =", ret["nMAC[6]"], ", Pid =", ret["nPortId"], ", nFId =", ret["nFId"], ", nAgeTimer =", ret["nAgeTimer"], )
    if int(ret["bLast"]) == 1:
        break
    ret = sw.pyGSW_MAC_TableEntryRead(0)
    

GSW_MAC_TableEntryRead    ---    Return value: GSW_statusOk 
bInitial = 0x0, 0 
bLast = 0x1, 1 
nFId = 0x0, 0 
nPortId = 0x0, 0 
nAgeTimer = 0x0, 0 
nSVLAN_Id = 0x0, 0 
bStaticEntry = 0x0, 0 
nMAC[6] = [ 0x00 0x00 0x00 0x00 0x00 0x00] 
ret =  GSW_statusOk
nMAC[6] = [ 0x00 0x00 0x00 0x00 0x00 0x00]  , Pid = 0 , nFId = 0 , nAgeTimer = 0


In [10]:
ret = sw.pyGSW_PortCfgGet(0)

GSW_PortCfgGet    ---    Return value: GSW_statusOk 
nPortId = 0x0, 0 
eEnable = 0x1, 1 GSW_PORT_ENABLE_RXTX  [GSW_portEnable_t]
bUnicastUnknownDrop = 0x0, 0 
bMulticastUnknownDrop = 0x0, 0 
bReservedPacketDrop = 0x0, 0 
bBroadcastDrop = 0x0, 0 
bAging = 0x0, 0 
bLearning = 0x0, 0 
bLearningMAC_PortLock = 0x0, 0 
nLearningLimit = 0xff, 255 
bMAC_SpoofingDetection = 0x0, 0 
eFlowCtrl = 0x1, 1 GSW_FLOW_RX  [GSW_portFlow_t]
ePortMonitor = 0x0, 0 GSW_PORT_MONITOR_NONE  [GSW_portMonitor_t]


In [7]:
pce = sw.clear_pce_data()
p = pce[0]
a = pce[1]
p["nIndex"] = 0
p["bEnable"] = 1
p["bPordIdEnable"] = 1
p["nPortId"] = 4
p["bDstIP_Select"] = 1
p["nDstIPipv4"] = 1*0x1000000
print (p["nDstIPipv4"])
p["nDstIP_Mask"] = 0x7e
a["ePortMapAction"] = 4
a["nForwardPortMap"] = 0x40
a["bPortBitMapMuxControl"] = 0
sw.pyGSW_PceRuleWrite(p, a)

16777216


{'ret': 0}

In [13]:
sw.pyGSW_PceRuleRead(1)

param.pattern.nDstIP.nIPv4 = 00000000, 0 
PCE rule index 1, enabled = 1
pattern:
   , bPortIdEnable = 0, nPortId = 0
   , bDSCP_Enable = 0, nDSCP = 0
   , bPCP_Enable = 0, nPCP = 0
   , bSTAG_PCP_DEI_Enable = 0, nSTAG_PCP_DEI = 0
   , bPktLngEnable = 0, nPktLng = 0, nPktLngRange = 0
   , bMAC_DstEnable = 0, nMAC_Dst = 00:00:00:00:00:00, nMAC_DstMask = 0
   , bMAC_SrcEnable = 0, nMAC_Src = 00:00:00:00:00:00, nMAC_SrcMask = 0
   , bAppDataMSB_Enable = 0, nAppDataMSB = 0
   , bAppMaskRangeMSB_Select = 0, nAppMaskRangeMSB = 0
   , bAppDataLSB_Enable = 0, nAppDataLSB = 0
   , bAppMaskRangeLSB_Select = 0, nAppMaskRangeLSB = 0
   , eDstIP_Select = 0, nDstIPipv4 = 0.0.0.0, nDstIPipv6 = 0000:0000:0000:0000, nDstIP_Mask = 0
   , eSrcIP_Select = 0, nSrcIPipv4 = 0.0.0.0, nSrcIPipv6 = 0000:0000:0000:0000, nSrcIP_Mask = 0
   , bEtherTypeEnable = 0, nEtherType = 0, nEtherTypeMask = 0
   , bProtocolEnable = 0, nProtocol = 0, nProtocolMask = 0
   , bSessionIdEnable = 0, nSessionId = 0, bVid = 0, nVid =

{'ret': 0}

In [7]:
pce = sw.clear_pce_data()
p = pce[0]
a = pce[1]
print (p)
def pyGSW_PceRuleWrite(p, a):
    print (p["bPortIdEnable"])
    print (a["ePortMapAction"])

pyGSW_PceRuleWrite(p, a)

{'nIndex': 0, 'bEnable': 0, 'bPortIdEnable': 0, 'nPortId': 0, 'bDSCP_Enable': 0, 'nDSCP': 0, 'bPCP_Enable': 0, 'nPCP': 0, 'bSTAG_PCP_DEI_Enable': 0, 'nSTAG_PCP_DEI': 0, 'bPktLngEnable': 0, 'nPktLng': 0, 'nPktLngRange': 0, 'bMAC_DstEnable': 0, 'nMAC_Dst0': 0, 'nMAC_Dst1': 0, 'nMAC_Dst2': 0, 'nMAC_Dst3': 0, 'nMAC_Dst4': 0, 'nMAC_Dst5': 0, 'nMAC_DstMask': 0, 'bMAC_SrcEnable': 0, 'nMAC_Src0': 0, 'nMAC_Src1': 0, 'nMAC_Src2': 0, 'nMAC_Src3': 0, 'nMAC_Src4': 0, 'nMAC_Src5': 0, 'nMAC_SrcMask': 0, 'bAppDataMSB_Enable': 0, 'nAppDataMSB': 0, 'bAppMaskRangeMSB_Select': 0, 'nAppMaskRangeMSB': 0, 'bAppDataLSB_Enable': 0, 'nAppDataLSB': 0, 'bAppMaskRangeLSB_Select': 0, 'nAppMaskRangeLSB': 0, 'eDstIP_Select': 0, 'nDstIPipv4': 0, 'nDstIPipv60': 0, 'nDstIPipv61': 0, 'nDstIPipv62': 0, 'nDstIPipv63': 0, 'nDstIPipv64': 0, 'nDstIPipv65': 0, 'nDstIPipv66': 0, 'nDstIPipv67': 0, 'nDstIP_Mask': 0, 'eSrcIP_Select': 0, 'nSrcIPipv4': 0, 'nSrcIPipv60': 0, 'nSrcIPipv61': 0, 'nSrcIPipv62': 0, 'nSrcIPipv63': 0, 'nSrcI

#### Global Switch Macro Hardware Reset

Reset all hardware modules including the register settings. This reset
acts similar to the hardware reset and is cleared by HW after Reset is
executed.
reg 0xF400 = 0x0002

In [31]:
a = "u8 act_p1, u6 act_p2, int pattern_p5"
a = a.split(", ")
c = list()
for b in a:
    c.append(b.split("_"))
print (c)
a = list()
for b in c:
    t = b[0].split(" ")
    t.append(b[1])
    a.append(t)
c = list()
for b in a:
    c.append(b[1] + " = " + b[2])
c
    

[['u8 act', 'p1'], ['u6 act', 'p2'], ['int pattern', 'p5']]


['act = p1', 'act = p2', 'pattern = p5']

In [5]:
s = ""
s = s + "	fo = open(fn, \"r\")\n	for line in fo.readlines():\n		if len(line.split()) == 1:\n			line = line[:-1] + \"    ---    Return value: \" + nRet[ret[\"ret\"]] + \"\\n\"\n		else:\n			if line.split()[0][-1] == \"]\":\n				s = \"\"\n				for i in line.split()[2:]:\n					s = s + i + \" \"\n				ret[line.split()[0]] = s\n			else:\n				ret[line.split()[0]] = line.split()[3]\n		if pr == 1:\n			s = \"\"\n			for st in swse.struct:\n				for p in swse.struct[st]:\n					if p[1] == line.split()[0]:\n						e = p[0]\n						if p[0] in swse.enum:\n							s = swse.enum[p[0]][line.split()[3]] + \"  [\" + p[0] + \"]\"\n			print (line[:-1], s)\n	fo.close()\n	return ret"
print (s)

	fo = open(fn, "r")
	for line in fo.readlines():
		if len(line.split()) == 1:
			line = line[:-1] + "    ---    Return value: " + nRet[ret["ret"]] + "\n"
		else:
			if line.split()[0][-1] == "]":
				s = ""
				for i in line.split()[2:]:
					s = s + i + " "
				ret[line.split()[0]] = s
			else:
				ret[line.split()[0]] = line.split()[3]
		if pr == 1:
			s = ""
			for st in swse.struct:
				for p in swse.struct[st]:
					if p[1] == line.split()[0]:
						e = p[0]
						if p[0] in swse.enum:
							s = swse.enum[p[0]][line.split()[3]] + "  [" + p[0] + "]"
			print (line[:-1], s)
	fo.close()
	return ret


#### Reconnect the serial port 
And enable the UART - MDIO bridge

In [16]:
sw.rc()

Now close port at: com24
Open serial port at: com24
('r(0xf383) = ', '0000')
('r(0xf384) = ', '003c')
('w(0xf383) = ', '0003')
('w(0xf384) = ', '003f')


#### Close the serial port

In [43]:
sw.s.close_port()

'Now close port at: com15'

#### Port summary

In [5]:
for a in range(5):
    sw.Phy_sum(a)
    print (" ")

Phy: 0    Link state: 0.0     Speed: 10 Mbps
Master(1)/Slave(0): 0.0     Local receiver OK: 0.0     Remote receiver OK: 0.0
SNR margin: 15.0     Cable length: 150 m


TypeError: list indices must be integers or slices, not float

#### Set the counter type for port n

In [25]:
sw.set_counter(0)

0: rx err
1: rx frames
2: ESD err
3: SSD err
4: tx err
5: tx frames
6: collision
7: no. of link down
8: no. of ADS
Which counter to set: 0-8 [1]1


### Port reditect

  #### CPU port set / get
    def pyGSW_CPU_PortCfgSet(pid, valid, ing, eg, fcsch, fcsgen, spectg):

    def pyGSW_CPU_PortCfgGet(pid):

  #### Port to be redirected set / get
    def pyGSW_PortRedirectSet(pid, eg, ing):

    def pyGSW_PortRedirectGet(pid):


In [5]:
sw.pyGSW_CPU_PortCfgSet(4, 1, 0, 0, 0, 0, 0)

{'ret': 0}

In [3]:
ret = sw.pyGSW_CPU_PortCfgGet(4)
ret

KeyError: 'value:'

In [29]:
sw.pyGSW_PortRedirectSet(0, 0, 1)
sw.pyGSW_PortRedirectSet(1, 0, 1)
sw.pyGSW_PortRedirectSet(2, 0, 1)
sw.pyGSW_PortRedirectSet(3, 0, 1)

{'ret': 0}

In [30]:
sw.pyGSW_PortRedirectGet(0)
sw.pyGSW_PortRedirectGet(1)
sw.pyGSW_PortRedirectGet(2)
sw.pyGSW_PortRedirectGet(3)

GSW_PortRedirectGet    ---    Return value: GSW_statusOk
nPortId = 0x0, 0
bRedirectEgress = 0x0, 0
bRedirectIngress = 0x1, 1
GSW_PortRedirectGet    ---    Return value: GSW_statusOk
nPortId = 0x1, 1
bRedirectEgress = 0x0, 0
bRedirectIngress = 0x1, 1
GSW_PortRedirectGet    ---    Return value: GSW_statusOk
nPortId = 0x2, 2
bRedirectEgress = 0x0, 0
bRedirectIngress = 0x1, 1
GSW_PortRedirectGet    ---    Return value: GSW_statusOk
nPortId = 0x3, 3
bRedirectEgress = 0x0, 0
bRedirectIngress = 0x1, 1


{'ret': 0, 'nPortId': '3', 'bRedirectEgress': '0', 'bRedirectIngress': '1'}

In [6]:
sw.pyGSW_PortRedirectSet(0, 0, 0)
sw.pyGSW_PortRedirectSet(1, 0, 0)
sw.pyGSW_PortRedirectSet(2, 0, 0)
sw.pyGSW_PortRedirectSet(3, 0, 0)

0

#### Port config set / get
def pyGSW_PortCfgSet(pid, en, uni, mul, res, brd, aging, learning, portlock, learninglimit, spoofdet, flowctl, portmon):

def pyGSW_PortCfgGet(pid):


In [33]:
#sw.pyGSW_PortCfgSet(0, 1, 0,0,0,0, 0,0,1, 0xff, 0, 0, 0)
#sw.pyGSW_PortCfgSet(1, 1, 0,0,0,0, 0,0,1, 0xff, 0, 0, 0)
#sw.pyGSW_PortCfgSet(2, 1, 0,0,0,0, 0,0,1, 0xff, 0, 0, 0)
#sw.pyGSW_PortCfgSet(3, 1, 0,0,0,0, 0,0,1, 0xff, 0, 0, 0)
sw.pyGSW_PortCfgSet(4, 1, 0,0,0,0, 0,0,1, 0xff, 0, 0, 0)


{'ret': 0}

In [6]:
sw.pyGSW_PortCfgSet(4, 1, 0,0,0,0, 0,0,0, 0xff, 0, 0, 0)

100

In [34]:
for p in range(5):
    sw.pyGSW_PortCfgGet(p)
    print ("")

GSW_PortCfgGet    ---    Return value: GSW_statusOk
nPortId = 0x0, 0
eEnable = 0x1, 1
bUnicastUnknownDrop = 0x0, 0
bMulticastUnknownDrop = 0x0, 0
bReservedPacketDrop = 0x0, 0
bBroadcastDrop = 0x0, 0
bAging = 0x0, 0
bLearning = 0x0, 0
bLearningMAC_PortLock = 0x0, 0
nLearningLimit = 0xff, 255
bMAC_SpoofingDetection = 0x0, 0
eFlowCtrl = 0x1, 1
ePortMonitor = 0x0, 0

GSW_PortCfgGet    ---    Return value: GSW_statusOk
nPortId = 0x1, 1
eEnable = 0x1, 1
bUnicastUnknownDrop = 0x0, 0
bMulticastUnknownDrop = 0x0, 0
bReservedPacketDrop = 0x0, 0
bBroadcastDrop = 0x0, 0
bAging = 0x0, 0
bLearning = 0x0, 0
bLearningMAC_PortLock = 0x0, 0
nLearningLimit = 0xff, 255
bMAC_SpoofingDetection = 0x0, 0
eFlowCtrl = 0x1, 1
ePortMonitor = 0x0, 0

GSW_PortCfgGet    ---    Return value: GSW_statusOk
nPortId = 0x2, 2
eEnable = 0x1, 1
bUnicastUnknownDrop = 0x0, 0
bMulticastUnknownDrop = 0x0, 0
bReservedPacketDrop = 0x0, 0
bBroadcastDrop = 0x0, 0
bAging = 0x0, 0
bLearning = 0x0, 0
bLearningMAC_PortLock = 0x0, 0
nLe

### Storm Control
#### The used APIs for storm control

sw.pyGSW_QoS_MeterCfgSet(en, mid, cbs, ebs, rate)

pyGSW_QoS_MeterPortAssign(mid, dir, piid, peid)  - dir:0 None, 1 Ingress, 2 Egress, 3 Both. 

pyGSW_QoS_StormCfgSet(mid, bc, mc, uc)

pyGSW_QoS_WredCfgSet(pf, tm, redmin, redmax, yellowmin, yellowmax, greenmin, greenmax)

In [17]:
#50Mbps
#sw.pyGSW_QoS_MeterCfgSet(1, 0, 500, 0, 50*1024)#port 0 ingress
#sw.pyGSW_QoS_MeterPortAssign(0, 1, 0, 0)

In [18]:
#sw.pyGSW_QoS_MeterCfgSet(1, 1, 500, 0, 50*1024)
#port 3 ingress
#sw.pyGSW_QoS_MeterPortAssign(0, 1, 3, 0)

In [8]:
#50Mbps
sw.pyGSW_QoS_MeterCfgSet(1, 0, 4000, 0, 5*1024)
sw.pyGSW_QoS_StormCfgSet(0, 1, 1, 1)
sw.pyGSW_QoS_WredCfgSet(0, 0, 0, 0, 0, 0, 0xffff, 0xffff)

0

#### 2 * EASY GPY211 back to back throughput test
Use 3 ports (port 0, 2, 3) of GWT140 as trunk to connect 2 * EASY GPY211. Configure:

**Board #0:** 

MAC_TableEntryAdd 0 0 1000000 0 1 0 00:22:A2:00:00:01

MAC_TableEntryAdd 0 2 1000000 0 1 0 00:22:A2:00:00:03

MAC_TableEntryAdd 0 3 1000000 0 1 0 00:22:A2:00:00:05

MAC_TableEntryAdd 0 4 1000000 0 1 0 00:22:A2:00:00:00

MAC_TableEntryAdd 0 4 1000000 0 1 0 00:22:A2:00:00:02

MAC_TableEntryAdd 0 4 1000000 0 1 0 00:22:A2:00:00:04

Traffic input from Port 4, the 2.5G PHY, des MAC are 00:22:A2:00:00:01/03/05

**Board #1:** 

MAC_TableEntryAdd 0 0 1000000 0 1 0 00:22:A2:00:00:00

MAC_TableEntryAdd 0 2 1000000 0 1 0 00:22:A2:00:00:02

MAC_TableEntryAdd 0 3 1000000 0 1 0 00:22:A2:00:00:04

MAC_TableEntryAdd 0 4 1000000 0 1 0 00:22:A2:00:00:01

MAC_TableEntryAdd 0 4 1000000 0 1 0 00:22:A2:00:00:03

MAC_TableEntryAdd 0 4 1000000 0 1 0 00:22:A2:00:00:05

Traffic input from Port 4, the 2.5G PHY, des MAC are 00:22:A2:00:00:00/02/04

pyGSW_MAC_TableEntryAdd(nFId, nPortId, nAgeTimer, nSVLAN_Id, bStaticEntry, nTrafficClass, nMAC_0, nMAC_1, nMAC_2, nMAC_3, nMAC_4, nMAC_5):


In [33]:
#Board #0
for a in range(4):
    sw.pyGSW_MAC_TableEntryAdd(0, 0, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x01+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 1, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x03+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 2, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x05+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 3, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x07+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 4, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x00+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 4, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x02+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 4, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x04+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 4, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x06+a*8)

In [6]:
#Board #1
for a in range(1):
    sw.pyGSW_MAC_TableEntryAdd(0, 0, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x00+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 1, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x02+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 2, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x04+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 3, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x06+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 4, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x01+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 4, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x03+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 4, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x05+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 4, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x07+a*8)

#### MAC Table read
| Device            | MAC addr          |
| ----------------- |:-----------------:|
| HP_Z2 I915        | 84:a9:3e:71:dd:95 |
| Tektronix MDO3054 | 08:00:11:21:f9:61 |
| DELL_YJCH_Home    | 14:fe:b5:b4:37:9f |
| 550XT2.p0         | b4:96:91:60:40:f0 |
| 550XT2.p1         | b4:96:91:60:40:f1 |
| Thinkpad 1G       | 98:fa:9b:a9:e6:b2 |


In [2]:
ret = sw.pyGSW_MAC_TableEntryRead(1)
for a in range(100):
    if ret == 1:
        break
    ret = sw.pyGSW_MAC_TableEntryRead(0)

TypeError: in method 'pyGSW_MAC_TableEntryRead', argument 1 of type 'ltq_bool_t'

In [19]:
sw.pyGSW_PortCfgSet(5, 1, 0,0,0,0, 0,0,1, 0xff, 0, 0, 0)


100

#### Play around

In [17]:
print (hex(sw.mdior(0, 0x15)))
print (hex(sw.mdior(0, 3)))

0x0
0xa409


In [34]:
sw.pyGSW_PceRuleReadf(0, 0)

[PySwitch pce_rule p.nIndex = 0, p.bEnable = 0 -- read]
------------------------------------------------------------------------
PCE rule index 0, enabled = 0
pattern:bPortIdEnable = 0, nPortId = 0
        bDSCP_Enable = 0, nDSCP = 0
        bPCP_Enable = 0, nPCP = 0
        bSTAG_PCP_DEI_Enable = 0, nSTAG_PCP_DEI = 0
        bPktLngEnable = 0, nPktLng = 0, nPktLngRange = 0
        bMAC_DstEnable = 0, nMAC_Dst = 00:00:00:00:00:00, nMAC_DstMask = 0x0
        bMAC_SrcEnable = 0, nMAC_Src = 00:00:00:00:00:00, nMAC_SrcMask = 0x0
        bAppDataMSB_Enable = 0, nAppDataMSB = 0, bAppMaskRangeMSB_Select = 0, nAppMaskRangeMSB = 0
        bAppDataLSB_Enable = 0, nAppDataLSB = 0, bAppMaskRangeLSB_Select = 0, nAppMaskRangeLSB = 0
        eDstIP_Select = 0, nDstIP = 00000000, nDstIP_Mask = 0x0
        eSrcIP_Select = 0, nSrcIP = 00000000, nSrcIP_Mask = 0x0
Action:
eTrafficClassAction = 0, nTrafficClassAlternate = 0
eSnoopingTypeAction = 0
eLearningAction = 0
eIrqAction = 0
eCrossStateAction = 0
eC

In [8]:
print (hex(sw.mdior(0, 9)))

0x0


In [7]:
sw.mdiow(0,9,0)

0

In [1]:
import ser02 as s
s.list_ports()
s.open_port("/dev/ttyUSB1")

module ser02.py: list_ports(), open_port("COMx"), r(addr), w(addr, value), close_ports()
device: /dev/ttyS0
name: ttyS0
description: n/a
device_path: /sys/devices/platform/soc/fe215040.serial
hwid: n/a
subsystem: platform

device: /dev/ttyUSB3
name: ttyUSB3
description: Quad RS232-HS
device_path: /sys/devices/platform/scb/fd500000.pcie/pci0000:00/0000:00:00.0/0000:01:00.0/usb1/1-1/1-1.3/1-1.3:1.3/ttyUSB3
hwid: USB VID:PID=0403:6011 LOCATION=1-1.3:1.3
subsystem: usb-serial

device: /dev/ttyUSB2
name: ttyUSB2
description: Quad RS232-HS
device_path: /sys/devices/platform/scb/fd500000.pcie/pci0000:00/0000:00:00.0/0000:01:00.0/usb1/1-1/1-1.3/1-1.3:1.2/ttyUSB2
hwid: USB VID:PID=0403:6011 LOCATION=1-1.3:1.2
subsystem: usb-serial

device: /dev/ttyUSB1
name: ttyUSB1
description: Quad RS232-HS
device_path: /sys/devices/platform/scb/fd500000.pcie/pci0000:00/0000:00:00.0/0000:01:00.0/usb1/1-1/1-1.3/1-1.3:1.1/ttyUSB1
hwid: USB VID:PID=0403:6011 LOCATION=1-1.3:1.1
subsystem: usb-serial

device: /dev

In [1]:
import ser02 as s
s.list_ports()
pn = input("Please input the port name (e.g. COMx): ")
print (pn)
s.open_port("/dev/ttyUSB1")


module ser02.py: list_ports(), open_port("COMx"), r(addr), w(addr, value), close_ports()
device: /dev/ttyS0
name: ttyS0
description: n/a
device_path: /sys/devices/platform/soc/fe215040.serial
hwid: n/a
subsystem: platform

device: /dev/ttyUSB3
name: ttyUSB3
description: Quad RS232-HS
device_path: /sys/devices/platform/scb/fd500000.pcie/pci0000:00/0000:00:00.0/0000:01:00.0/usb1/1-1/1-1.3/1-1.3:1.3/ttyUSB3
hwid: USB VID:PID=0403:6011 LOCATION=1-1.3:1.3
subsystem: usb-serial

device: /dev/ttyUSB2
name: ttyUSB2
description: Quad RS232-HS
device_path: /sys/devices/platform/scb/fd500000.pcie/pci0000:00/0000:00:00.0/0000:01:00.0/usb1/1-1/1-1.3/1-1.3:1.2/ttyUSB2
hwid: USB VID:PID=0403:6011 LOCATION=1-1.3:1.2
subsystem: usb-serial

device: /dev/ttyUSB1
name: ttyUSB1
description: Quad RS232-HS
device_path: /sys/devices/platform/scb/fd500000.pcie/pci0000:00/0000:00:00.0/0000:01:00.0/usb1/1-1/1-1.3/1-1.3:1.1/ttyUSB1
hwid: USB VID:PID=0403:6011 LOCATION=1-1.3:1.1
subsystem: usb-serial

device: /dev

In [2]:
print ("0x%04x"%s.r(0xfa10))

r fa10
1713
>
0x1713 <class 'int'>
press any key to continue 16
0x1713


In [10]:
s.ser.write(b'\x1B')

1

In [11]:
s.rev_anything("")


>


KeyboardInterrupt: 

In [8]:
b'\x80abc'.decode("utf-8", "backslashreplace")

'\\x80abc'

In [13]:
str.encode(">")[0]

62

In [1]:
pn = input("Please input the port name (e.g. COMx): ")
print (pn)


Please input the port name (e.g. COMx): /dev/ttyUSB1
/dev/ttyUSB1


In [2]:
import test as t

device: /dev/ttyS0
name: ttyS0
description: n/a
device_path: /sys/devices/platform/soc/fe215040.serial
hwid: n/a
subsystem: platform

device: /dev/ttyUSB3
name: ttyUSB3
description: Quad RS232-HS
device_path: /sys/devices/platform/scb/fd500000.pcie/pci0000:00/0000:00:00.0/0000:01:00.0/usb1/1-1/1-1.3/1-1.3:1.3/ttyUSB3
hwid: USB VID:PID=0403:6011 LOCATION=1-1.3:1.3
subsystem: usb-serial

device: /dev/ttyUSB2
name: ttyUSB2
description: Quad RS232-HS
device_path: /sys/devices/platform/scb/fd500000.pcie/pci0000:00/0000:00:00.0/0000:01:00.0/usb1/1-1/1-1.3/1-1.3:1.2/ttyUSB2
hwid: USB VID:PID=0403:6011 LOCATION=1-1.3:1.2
subsystem: usb-serial

device: /dev/ttyUSB1
name: ttyUSB1
description: Quad RS232-HS
device_path: /sys/devices/platform/scb/fd500000.pcie/pci0000:00/0000:00:00.0/0000:01:00.0/usb1/1-1/1-1.3/1-1.3:1.1/ttyUSB1
hwid: USB VID:PID=0403:6011 LOCATION=1-1.3:1.1
subsystem: usb-serial

device: /dev/ttyUSB0
name: ttyUSB0
description: Quad RS232-HS
device_path: /sys/devices/platform/scb/

In [2]:
b = 5
print (type(b))


<class 'int'>
